# Workbench

The goal of this project is to develop a PyShiny app capable of visualising the following for a selected neighbourhood of Boston, MA:

1. The most common **destination** for cyclists renting a BlueBike in that neighbourhood
2. The most common **origin** for cyclists who rented a BlueBike elsewhere, and finished their journey in the selected neighborhood
3. Over the course of the day, the times at which the most cyclists *arrived* in the neighbourhood by bicycle
4. Over the course of the day, the times at which the most cyclists *departed* the neighbourhood by bicycle

Ideally, the following variables will be selectable in the user input:
- The neighbourhood
- The direction (inbound vs. outbound)

The data processing will be carried out using a local instance of PySpark

## 1. Data engineering and processing
### 1. Setting up Spark

We start by setting up a PySpark cluster

In [1]:
#Reference: https://sparkbyexamples.com/spark/how-to-create-a-sparksession-and-spark-context/
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

We then check that it runs correctly

In [ ]:
print("First SparkContext:");
print("APP Name :"+spark.sparkContext.appName);
print("Master :"+spark.sparkContext.master);

### 2. Developing the BlueBike data access code locally

#### 1. Ridership data
First, we configure the environment zip path variables

The first approach was not successful

It was necessary to use the load_dotenv package:

In [ ]:
#Location: https://stackoverflow.com/questions/63019506/python-get-value-of-env-variable-from-a-specific-env-file
from dotenv import load_dotenv
load_dotenv()
import os
x = os.getenv("zip_path")
print(x)

Next, we download and unzip the file:

In [ ]:
! wget https://s3.amazonaws.com/hubway-data/202211-bluebikes-tripdata.zip -P data/


In [ ]:
#https://stackoverflow.com/questions/3451111/unzipping-files-in-python
import zipfile
with zipfile.ZipFile(os.getenv("zip_path"), 'r') as zip_ref:
    zip_ref.extractall("data/")

Inspecting the data:

In [ ]:
df = spark.read.csv("data/202211-bluebikes-tripdata.csv", header=True)
df.printSchema()
df.head()

#### 2. Station data
It was also necessary to match the stations with Boston's various neighbourhoods.


In [ ]:
import pyogrio

In [ ]:
import pandas as pd
pd.read_csv(os.getenv("station_path"), header=1).head()

In [ ]:
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file
import geopandas
import pandas as pd

# Boston neighbourhoods
polydf = read_file('data/Boston_Neighborhoods.geojson')

stations = pd.read_csv(os.getenv("station_path"), header=1)
pointdf = GeoDataFrame(
    stations, geometry=geopandas.points_from_xy(stations.Longitude, stations.Latitude))

pointdf.set_crs(epsg='4326', inplace=True)

# Make sure they're using the same projection reference

In [ ]:
#https://geopandas.org/en/stable/gallery/spatial_joins.html
joined_df = pointdf.sjoin(polydf, how="left")
grab_df = joined_df[['Name_left', 'Name_right', 'District']]
import sqldf
matched_pairs = sqldf.run('SELECT DISTINCT Name_left as Station, Name_right as Neighbourhood from grab_df where District = \'Boston\'')
matched_pairs.to_csv('data/neighbourhood_stations.csv')
matched_pairs.head()

In [ ]:
grab_df

### 3. Calling the ETL classes

In [ ]:
from sparkbike.etl import ExtractTransformLoad
worker = ExtractTransformLoad()
worker.zip()
worker.geojoin()

### 4. PostgreSQL connection


In [2]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
from sqlalchemy import create_engine
alchemyEngine=create_engine(os.getenv('postgresql_path'), pool_recycle=3600)
dbConnection=alchemyEngine.connect()
stations=pd.read_sql("SELECT * FROM stations;", dbConnection)
stations

,a,Station,Neighbourhood
0,2.0,175 N Harvard St,Allston
1,4.0,2 Hummingbird Lane at Olmsted Green,Mattapan
2,8.0,555 Metropolitan Ave,Hyde Park
3,9.0,606 American Legion Hwy at Canterbury St,Roslindale
4,10.0,645 Summer St,South Boston Waterfront
...,...,...,...
245,441.0,Wentworth Institute of Technology - Huntington...,Fenway
246,442.0,West End Park,West End
247,444.0,Western Ave at Richardson St,Brighton
248,445.0,Whittier St Health Center,Roxbury


In [4]:
journeys=pd.read_sql("SELECT * FROM journeys;", dbConnection)
journeys

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code
0,803.0,2022-11-01 00:00:04,2022-11-01 00:13:28,46.0,Christian Science Plaza - Massachusetts Ave at...,42.343666,-71.085824,472.0,MIT Hayward St at Amherst St,42.360859,-71.085519,2088.0,Subscriber,02142
1,759.0,2022-11-01 00:00:23,2022-11-01 00:13:02,19.0,Park Dr at Buswell St,42.347241,-71.105301,52.0,Newbury St at Hereford St,42.348717,-71.085954,6889.0,Customer,None
2,1314.0,2022-11-01 00:00:55,2022-11-01 00:22:50,459.0,Canal St at Causeway St,42.365301,-71.060922,404.0,Mass Ave T Station,42.341356,-71.083370,3099.0,Customer,19454
3,541.0,2022-11-01 00:01:09,2022-11-01 00:10:11,30.0,Brigham Circle - Francis St at Huntington Ave,42.334629,-71.104079,5.0,Northeastern University - North Parking Lot,42.341814,-71.090179,7374.0,Subscriber,08802
4,478.0,2022-11-01 00:01:29,2022-11-01 00:09:27,554.0,Forsyth St at Huntington Ave,42.339202,-71.090511,360.0,Bartlett St at John Elliot Sq,42.329463,-71.090158,5108.0,Subscriber,02115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290616,430.0,2022-11-30 23:54:03,2022-12-01 00:01:13,49.0,Stuart St at Charles St,42.351146,-71.066289,47.0,Cross St at Hanover St,42.362811,-71.056067,5316.0,Subscriber,02113
290617,483.0,2022-11-30 23:55:32,2022-12-01 00:03:35,97.0,Harvard University River Houses at DeWolfe St ...,42.369239,-71.116870,104.0,Harvard University Radcliffe Quadrangle at She...,42.380287,-71.125107,6920.0,Customer,02138
290618,265.0,2022-11-30 23:57:44,2022-12-01 00:02:09,104.0,Harvard University Radcliffe Quadrangle at She...,42.380287,-71.125107,74.0,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,7638.0,Subscriber,22101
290619,327.0,2022-11-30 23:58:33,2022-12-01 00:04:00,12.0,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd,42.336244,-71.087986,200.0,Washington St at Melnea Cass Blvd,42.332817,-71.081198,8088.0,Subscriber,02119


In [11]:
journeys=pd.read_sql("SELECT \"start station name\" FROM journeys;", dbConnection)
journeys

,start station name
0,Christian Science Plaza - Massachusetts Ave at...
1,Park Dr at Buswell St
2,Canal St at Causeway St
3,Brigham Circle - Francis St at Huntington Ave
4,Forsyth St at Huntington Ave
...,...
290616,Stuart St at Charles St
290617,Harvard University River Houses at DeWolfe St ...
290618,Harvard University Radcliffe Quadrangle at She...
290619,Ruggles T Stop - Columbus Ave at Melnea Cass Blvd


In [14]:
enriched_journeys = pd.read_sql("SELECT stations.Neighbourhood, count(journeys.*) as total_journeys \
                                 FROM stations \
                                 LEFT JOIN journeys \
                                ON stations.Station = journeys.\"start station name\";", 
                                dbConnection)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column stations.station does not exist
LINE 1: ... JOIN journeys                                 ON stations.S...
                                                             ^
HINT:  Perhaps you meant to reference the column "stations.Station".

[SQL: SELECT stations.Neighbourhood, count(journeys.*) as total_journeys                                  FROM stations                                  LEFT JOIN journeys                                 ON stations.Station = journeys."start station name";]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [2]:
from sparkbike.sqlconnector import sqlconnector
sqlconnector("SELECT 1 FROM stations;")

,?column?
0,1
1,1
2,1
3,1
4,1
...,...
245,1
246,1
247,1
248,1


## 2. PyShiny development

In [ ]:
import os
os.system('shiny create delta_app')


## 3. Airflow development
- Install Airflow
- Then change config to point to local dags setup (this should be part of setup file)
- Then launch/create database